# gearstrategy_pipeline

**Short Introduction**  
The `gearstrategy_pipeline` function is designed to **analyze on-road driving data** and compare it to **laboratory test (dynamometer) data** for vehicle gear-change strategies. Specifically, it extracts the **threshold boundaries** for three different domains, **rear motor torque**, **power** output, and **accelerator pedal** position domain, by performing clustering, spline fitting, and subsequent comparisons. This enables the visualization and quantification of real-driving data derived gear-change strategy compared to a known or reference strategy derived under laboratory conditions.

---

## Parameters

- **data_cache** (`dict`, optional)  
  Provides pre-loaded data instead of reading from files. This option should be used when performing grid searches to avoid redundant file reads. Must contain:
  - `'results_normal_loaded'` and `'results_sport_loaded'` with reference lab data.  
  - `'normal_data'` and `'sport_data'` with raw driving data.

- **smoothing_kwargs** (`dict`, optional)  
  Arguments controlling time-domain smoothing of signals (used if `columns_to_smooth` is provided).
  Options:
  - `None`
  - `{'filter_type': 'moving_average', 'window_size': 3}`
  - `{'filter_type': 'exponential_moving_average', 'alpha': 0.4}`
  - `{'filter_type': 'savitzky_golay', 'window_size': 5, 'polyorder': 2}`
  - `{'filter_type': 'lowpass',  'cutoff_frequency': 0.1, 'order': 2}`

- **columns_to_smooth** (`list`, optional)  
  Which columns in the DataFrame to smooth (e.g., `['vehicle_speed', 'engine_rpm']`).

- **gear_change_offset_samples** (`int`, default=0)  
  Index offset to apply when labeling gear-change events in the CAN logs.

- **outlier_eps** (`float`, default=0.1)**  
  DBSCAN’s maximum neighborhood distance for detecting outliers.

- **outlier_min_samples** (`int`, default=1)**  
  DBSCAN’s minimum number of samples within a neighborhood to qualify as a core point.

- **close_points_merge_thr** (`float`, default=0)  
  Threshold for merging points that are very close in the normalized `(x, y)` space.

- **n_clusters** (`int`, default=10)  
  Number of clusters for k-means in each domain.

- **attach_endpoints** (`bool`, default=True)  
  Forces the clusters/spline to include boundary endpoints.

- **anchor_points** (`bool`, default=False)  
  If `True`, puts heavy weight on start and end point to effectively anchor points in the final spline.

- **spline_order** (`int`, default=1)  
  Polynomial order for the parametric spline.

- **num_knots** (`int`, default=5)  
  Number of knots used in the spline-fitting.

- **cluster_weight** (`float`, default=1)  
  Weight for cluster centers in the spline-fitting process.

- **normal_weight** (`float`, optional)  
  Weight for “normal” points in spline-fitting. Defaults to `1 - cluster_weight` if `None`.

- **knot_distr_method** (`str`, default='cluster')  
  Method for distributing knots (options: `'equidistant'`, `'quantile'`, `'cluster'`).

- **verbose** (`bool`, default=False)  
  If `True`, prints additional logs and intermediate results.

- **generate_comparison_plots** (`bool`, default=True)  
  If `True`, produces plots that compare on-road data vs. lab-reference curves.

- **generate_plots** (`bool`, default=False)  
  If `True`, plots intermediate steps (e.g., outlier detection, clustering, spline fits).

- **filename_prefix** (`str`, optional)  
  Prefix for naming the figures that get saved (PDF format).

- **figsize_1**, **figsize_2**, **figsize_3** (`tuple`, default=(10, 8))  
  Different figure sizes for various types of plots.

---

## Return Values

A tuple of metrics for **Normal** and **Sport** modes across the three domains, including:
1. **Hausdorff Distance**  
2. **Fréchet Distance**  
3. **DTW Distance**
4. **Chamfer Distance**  
5. **Parametric RMSE**  

These are returned in a fixed order for each domain in both modes, plus average RMSE values (Normal mode, Sport mode, and a global mean).

In [ ]:
from modules.parametric_pipelines import gearstrategy_pipeline

data_cache = None # caching not required

smoothing_kwargs = None
columns_to_smooth = None
gear_change_offset_samples = 0
outlier_eps = 0.05
outlier_min_samples = 2
close_points_merge_thr = 0.005
n_clusters = 5
attach_endpoints = True
anchor_points = True
spline_order = 1
num_knots = 5
cluster_weight = 0.3
normal_weight = None
knot_distr_method = 'cluster'

verbose = False
generate_plots = True
generate_comparison_plots = True
filename_prefix = 'filename_prefix'  # Adjust as needed
figsize_1 = (5, 5)
figsize_2 = (9, 5)
figsize_3 = (5, 5)

results = gearstrategy_pipeline(
    data_cache=data_cache,
    smoothing_kwargs=smoothing_kwargs,
    columns_to_smooth=columns_to_smooth,
    gear_change_offset_samples=gear_change_offset_samples,
    outlier_eps=outlier_eps,
    outlier_min_samples=outlier_min_samples,
    close_points_merge_thr=close_points_merge_thr,
    n_clusters=n_clusters,
    attach_endpoints=attach_endpoints,
    anchor_points=anchor_points,
    spline_order=spline_order,
    num_knots=num_knots,
    cluster_weight=cluster_weight,
    normal_weight=normal_weight,
    knot_distr_method=knot_distr_method,
    verbose=verbose,
    generate_plots=generate_plots,
    generate_comparison_plots=generate_comparison_plots,
    filename_prefix=filename_prefix,
    figsize_1=figsize_1,
    figsize_2=figsize_2,
    figsize_3=figsize_3
)

hd_rmt_normal, fd_rmt_normal, dtw_rmt_normal, cd_rmt_normal, prmse_rmt_normal, hd_pm_normal, fd_pm_normal, dtw_pm_normal, cd_pm_normal, prmse_pm_normal, hd_ap_normal, fd_ap_normal, dtw_ap_normal, cd_ap_normal, prmse_ap_normal, prmse_normal, hd_rmt_sport, fd_rmt_sport, dtw_rmt_sport, cd_rmt_sport, prmse_rmt_sport, hd_pm_sport, fd_pm_sport, dtw_pm_sport, cd_pm_sport, prmse_pm_sport, hd_ap_sport, fd_ap_sport, dtw_ap_sport, cd_ap_sport, prmse_ap_sport, prmse_sport, prmse_global = results

print('global mean rmse:', prmse_global)